# Tabnet 1 - Modelo antes de entrar a la universidad

In [1]:
!pip install pytorch-tabnet
!pip install tabnet
!pip install wget

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
dfl=pd.read_csv("C:/Users/Daniela Hernandez/dat_tabnet_2_pre.txt",header=0, sep=",",na_values='\t', encoding='latin-1')
#df['GRAD'] = df.GRAD.map(dict(Y=1, N=0))
#dfl.drop(["Rel_School_GPA", "GPA_1ER_ANO"], axis = 1, inplace = True)

In [5]:
dfl

,GRAD,Year,Faculty,Major,Apt_Verbal,Aprov_Matem,Apt_Matem,Aprov_Espanol,Aprov_Ingles,INGRESO_FAMILIAR,EDUC_PADRE,EDUC_MADRE,Gender,School_Type,Rel_Stud_GPA
0,N,2000,Facultad de Artes y Ciencias - Artes,EFEN,530,444,447,453,389,I1_L12.5,HS,HS,M,Private,0.996736
1,Y,2000,Facultad de Artes y Ciencias - Artes,CIPO,571,640,541,481,398,I1_L12.5,LHS,LHS,M,Public,1.073917
2,N,2000,Facultad de Artes y Ciencias - Ciencias,CIFI,544,630,594,500,369,I1_L12.5,LHS,HS,F,Public,0.880744
3,Y,2000,Facultad de Administración de Empresas,ADOF,541,547,479,584,544,I2_B12.5A20,LHS,Assoc or Less,F,Public,1.017136
4,N,2000,Facultad de Ciencias Agrícolas,AGGE,686,610,757,340,630,I1_L12.5,Grad,Grad,M,Private,1.037716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18408,N,1999,Facultad de Artes y Ciencias - Ciencias,CCOM,612,601,618,572,526,I4_B30A50,HS,Grad,M,Private,0.739071
18409,N,1999,Facultad de Artes y Ciencias - Ciencias,QUIM,550,584,565,500,452,I3_B20A30,Bachillerato,LHS,F,Public,1.050959
18410,Y,1999,Facultad de Ciencias Agrícolas,AGGE,533,646,660,536,462,I2_B12.5A20,LHS,LHS,F,Public,1.087183
18411,N,1999,Facultad de Artes y Ciencias - Ciencias,EDMA,533,575,649,410,499,I1_L12.5,LHS,Assoc or Less,M,Public,1.072113


In [3]:
target = 'GRAD'
if "Set" not in dfl.columns:
    dfl["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(dfl.shape[0],))

train_indices = dfl[dfl.Set=="train"].index
valid_indices = dfl[dfl.Set=="valid"].index
test_indices = dfl[dfl.Set=="test"].index

In [5]:
dfl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18413 entries, 0 to 18412
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GRAD              18413 non-null  object 
 1   Year              18413 non-null  int64  
 2   Faculty           18413 non-null  object 
 3   Major             18413 non-null  object 
 4   Apt_Verbal        18413 non-null  int64  
 5   Aprov_Matem       18413 non-null  int64  
 6   Apt_Matem         18413 non-null  int64  
 7   Aprov_Espanol     18413 non-null  int64  
 8   Aprov_Ingles      18413 non-null  int64  
 9   INGRESO_FAMILIAR  18413 non-null  object 
 10  EDUC_PADRE        18413 non-null  object 
 11  EDUC_MADRE        18413 non-null  object 
 12  Gender            18413 non-null  object 
 13  School_Type       18413 non-null  object 
 14  Rel_Stud_GPA      18413 non-null  float64
 15  Set               18413 non-null  object 
dtypes: float64(1), int64(6), object(9)
memor

In [6]:
nunique = dfl.nunique()
types = dfl.dtypes

categorical_columns = []
categorical_dims =  {}
for col in dfl.columns:
    if types[col] == 'object' or nunique[col] < 200:  #The nunique() function is used to count distinct observations over requested axis.
        print(col, dfl[col].nunique())
        l_enc = LabelEncoder()
        dfl[col] = dfl[col].fillna("VV_likely") 
        dfl[col] = l_enc.fit_transform(dfl[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        dfl.fillna(dfl.loc[train_indices, col].mean(), inplace=True)

GRAD 2
Year 12
Faculty 5
Major 54
INGRESO_FAMILIAR 5
EDUC_PADRE 6
EDUC_MADRE 6
Gender 2
School_Type 3
Set 3


In [7]:
dfl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18413 entries, 0 to 18412
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GRAD              18413 non-null  int32  
 1   Year              18413 non-null  int64  
 2   Faculty           18413 non-null  int32  
 3   Major             18413 non-null  int32  
 4   Apt_Verbal        18413 non-null  int64  
 5   Aprov_Matem       18413 non-null  int64  
 6   Apt_Matem         18413 non-null  int64  
 7   Aprov_Espanol     18413 non-null  int64  
 8   Aprov_Ingles      18413 non-null  int64  
 9   INGRESO_FAMILIAR  18413 non-null  int32  
 10  EDUC_PADRE        18413 non-null  int32  
 11  EDUC_MADRE        18413 non-null  int32  
 12  Gender            18413 non-null  int32  
 13  School_Type       18413 non-null  int32  
 14  Rel_Stud_GPA      18413 non-null  float64
 15  Set               18413 non-null  int32  
dtypes: float64(1), int32(9), int64(6)
memory

In [8]:
unused_feat = ['Set']

features = [ col for col in dfl.columns if col not in unused_feat+[target]] #Select features without target and Set.

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns] #Select features without target and Set in "features" is categorical.

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns] #Select features without target and Set in "features" is categorical with count distinct observations.

In [9]:
dfl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18413 entries, 0 to 18412
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GRAD              18413 non-null  int32  
 1   Year              18413 non-null  int64  
 2   Faculty           18413 non-null  int32  
 3   Major             18413 non-null  int32  
 4   Apt_Verbal        18413 non-null  int64  
 5   Aprov_Matem       18413 non-null  int64  
 6   Apt_Matem         18413 non-null  int64  
 7   Aprov_Espanol     18413 non-null  int64  
 8   Aprov_Ingles      18413 non-null  int64  
 9   INGRESO_FAMILIAR  18413 non-null  int32  
 10  EDUC_PADRE        18413 non-null  int32  
 11  EDUC_MADRE        18413 non-null  int32  
 12  Gender            18413 non-null  int32  
 13  School_Type       18413 non-null  int32  
 14  Rel_Stud_GPA      18413 non-null  float64
 15  Set               18413 non-null  int32  
dtypes: float64(1), int32(9), int64(6)
memory

In [6]:
dfl[features] = dfl[features].apply(lambda x: 1/(1+ np.exp(-x)), axis=0)  #Softmax Normalization

X_train = dfl[features].values[train_indices]
y_train = dfl[target].values[train_indices]

X_valid = dfl[features].values[valid_indices]
y_valid = dfl[target].values[valid_indices]

X_test = dfl[features].values[test_indices]
y_test = dfl[target].values[test_indices]

# Tuning

In [47]:
#hyperparameters

#optimizer_params_list=dict(lr=[2e-2,1])

max_epochs_list = [20,21,22,23,24,25,26,27,28,29,30]
batch_size_list = [1024,1025,1026,1027,1028,1029,1030]
virtual_batch_size_list = [128, 129, 130,131, 132,133, 134]
patience_list = [20,21,22,23,24,25,26]

In [48]:
def tabnet_grid_search(max_epochs,batch_size,virtual_batch_size,patience):
  model = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=1
                      )
  model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=patience,
    batch_size=batch_size, virtual_batch_size=virtual_batch_size,
    num_workers=0,
    weights=1,
    drop_last=False
    )
  Report = classification_report(y_test,model.predict(X_test),labels=[0,1], output_dict=True)
  return ([max_epochs,batch_size,virtual_batch_size,patience,Report['0']['recall']])

In [49]:
#Loop
result_list = []

for max_epochs in max_epochs_list:
  for batch_size in batch_size_list:
    for virtual_batch_size in virtual_batch_size_list:
      for patience in patience_list:
        result_list.append(tabnet_grid_search(max_epochs,batch_size,virtual_batch_size,patience))

Result_tabnet = pd.DataFrame(result_list, columns=['max_epochs','batch_size','virtual_batch_size','patience', 'Recall'])

Device used : cpu
epoch 0  | loss: 0.7404  | train_auc: 0.52182 | valid_auc: 0.5413  |  0:00:03s
epoch 1  | loss: 0.6955  | train_auc: 0.52265 | valid_auc: 0.53776 |  0:00:06s
epoch 2  | loss: 0.69378 | train_auc: 0.48281 | valid_auc: 0.46248 |  0:00:09s
epoch 3  | loss: 0.69326 | train_auc: 0.47691 | valid_auc: 0.45846 |  0:00:12s
epoch 4  | loss: 0.69253 | train_auc: 0.47811 | valid_auc: 0.45804 |  0:00:15s
epoch 5  | loss: 0.6933  | train_auc: 0.49735 | valid_auc: 0.50595 |  0:00:19s
epoch 6  | loss: 0.69384 | train_auc: 0.52972 | valid_auc: 0.53848 |  0:00:22s
epoch 7  | loss: 0.69244 | train_auc: 0.5229  | valid_auc: 0.54225 |  0:00:25s
epoch 8  | loss: 0.69287 | train_auc: 0.52334 | valid_auc: 0.54593 |  0:00:28s
epoch 9  | loss: 0.69185 | train_auc: 0.52146 | valid_auc: 0.54066 |  0:00:32s
epoch 10 | loss: 0.69236 | train_auc: 0.52312 | valid_auc: 0.54344 |  0:00:35s
epoch 11 | loss: 0.6925  | train_auc: 0.4753  | valid_auc: 0.46637 |  0:00:38s
epoch 12 | loss: 0.69189 | train_a

C:\Users\DANIELA HERNANDEZ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Device used : cpu
epoch 0  | loss: 0.7404  | train_auc: 0.52182 | valid_auc: 0.5413  |  0:00:03s
epoch 1  | loss: 0.6955  | train_auc: 0.52265 | valid_auc: 0.53776 |  0:00:06s
epoch 2  | loss: 0.69378 | train_auc: 0.48281 | valid_auc: 0.46248 |  0:00:09s
epoch 3  | loss: 0.69326 | train_auc: 0.47691 | valid_auc: 0.45846 |  0:00:12s
epoch 4  | loss: 0.69253 | train_auc: 0.47811 | valid_auc: 0.45804 |  0:00:15s
epoch 5  | loss: 0.6933  | train_auc: 0.49735 | valid_auc: 0.50595 |  0:00:18s
epoch 6  | loss: 0.69384 | train_auc: 0.52972 | valid_auc: 0.53848 |  0:00:22s
epoch 7  | loss: 0.69244 | train_auc: 0.5229  | valid_auc: 0.54225 |  0:00:25s
epoch 8  | loss: 0.69287 | train_auc: 0.52334 | valid_auc: 0.54593 |  0:00:28s
epoch 9  | loss: 0.69185 | train_auc: 0.52146 | valid_auc: 0.54066 |  0:00:31s
epoch 10 | loss: 0.69236 | train_auc: 0.52312 | valid_auc: 0.54344 |  0:00:34s
epoch 11 | loss: 0.6925  | train_auc: 0.4753  | valid_auc: 0.46637 |  0:00:37s
epoch 12 | loss: 0.69189 | train_a

C:\Users\DANIELA HERNANDEZ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Device used : cpu
epoch 0  | loss: 0.7404  | train_auc: 0.52182 | valid_auc: 0.5413  |  0:00:02s
epoch 1  | loss: 0.6955  | train_auc: 0.52265 | valid_auc: 0.53776 |  0:00:06s
epoch 2  | loss: 0.69378 | train_auc: 0.48281 | valid_auc: 0.46248 |  0:00:09s
epoch 3  | loss: 0.69326 | train_auc: 0.47691 | valid_auc: 0.45846 |  0:00:12s
epoch 4  | loss: 0.69253 | train_auc: 0.47811 | valid_auc: 0.45804 |  0:00:15s
epoch 5  | loss: 0.6933  | train_auc: 0.49735 | valid_auc: 0.50595 |  0:00:18s
epoch 6  | loss: 0.69384 | train_auc: 0.52972 | valid_auc: 0.53848 |  0:00:21s
epoch 7  | loss: 0.69244 | train_auc: 0.5229  | valid_auc: 0.54225 |  0:00:25s
epoch 8  | loss: 0.69287 | train_auc: 0.52334 | valid_auc: 0.54593 |  0:00:27s
epoch 9  | loss: 0.69185 | train_auc: 0.52146 | valid_auc: 0.54066 |  0:00:30s
epoch 10 | loss: 0.69236 | train_auc: 0.52312 | valid_auc: 0.54344 |  0:00:33s
epoch 11 | loss: 0.6925  | train_auc: 0.4753  | valid_auc: 0.46637 |  0:00:36s
epoch 12 | loss: 0.69189 | train_a

...

C:\Users\DANIELA HERNANDEZ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Device used : cpu
epoch 0  | loss: 0.72873 | train_auc: 0.52135 | valid_auc: 0.54242 |  0:00:03s
epoch 1  | loss: 0.69942 | train_auc: 0.53082 | valid_auc: 0.54845 |  0:00:06s
epoch 2  | loss: 0.69435 | train_auc: 0.52206 | valid_auc: 0.53363 |  0:00:09s
epoch 3  | loss: 0.69372 | train_auc: 0.52188 | valid_auc: 0.54098 |  0:00:12s
epoch 4  | loss: 0.69297 | train_auc: 0.52315 | valid_auc: 0.54582 |  0:00:15s
epoch 5  | loss: 0.69329 | train_auc: 0.52143 | valid_auc: 0.52838 |  0:00:19s
epoch 6  | loss: 0.69293 | train_auc: 0.5247  | valid_auc: 0.54564 |  0:00:22s
epoch 7  | loss: 0.69288 | train_auc: 0.52769 | valid_auc: 0.53404 |  0:00:25s
epoch 8  | loss: 0.69255 | train_auc: 0.52118 | valid_auc: 0.5277  |  0:00:28s
epoch 9  | loss: 0.69193 | train_auc: 0.52874 | valid_auc: 0.53795 |  0:00:31s
epoch 10 | loss: 0.69134 | train_auc: 0.47168 | valid_auc: 0.46552 |  0:00:35s
epoch 11 | loss: 0.69402 | train_auc: 0.51399 | valid_auc: 0.52901 |  0:00:38s
epoch 12 | loss: 0.69215 | train_a

C:\Users\DANIELA HERNANDEZ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Device used : cpu
epoch 0  | loss: 0.73408 | train_auc: 0.49978 | valid_auc: 0.49953 |  0:00:02s
epoch 1  | loss: 0.69791 | train_auc: 0.47656 | valid_auc: 0.45408 |  0:00:05s
epoch 2  | loss: 0.69408 | train_auc: 0.52189 | valid_auc: 0.54197 |  0:00:08s
epoch 3  | loss: 0.69438 | train_auc: 0.47237 | valid_auc: 0.46242 |  0:00:11s
epoch 4  | loss: 0.69293 | train_auc: 0.4701  | valid_auc: 0.46153 |  0:00:15s
epoch 5  | loss: 0.69321 | train_auc: 0.47045 | valid_auc: 0.46234 |  0:00:18s
epoch 6  | loss: 0.69227 | train_auc: 0.47716 | valid_auc: 0.45368 |  0:00:21s
epoch 7  | loss: 0.69216 | train_auc: 0.47645 | valid_auc: 0.45131 |  0:00:23s
epoch 8  | loss: 0.69216 | train_auc: 0.47457 | valid_auc: 0.46666 |  0:00:26s
epoch 9  | loss: 0.69174 | train_auc: 0.52895 | valid_auc: 0.53438 |  0:00:30s
epoch 10 | loss: 0.69178 | train_auc: 0.525   | valid_auc: 0.54087 |  0:00:33s
epoch 11 | loss: 0.69235 | train_auc: 0.52909 | valid_auc: 0.55477 |  0:00:35s
epoch 12 | loss: 0.69176 | train_a

KeyboardInterrupt: 

In [50]:
Result_tabnet = pd.DataFrame(result_list, columns=['max_epochs','batch_size','virtual_batch_size','patience', 'Recall'])
Result_tabnet.sort_values(by=['Recall'], inplace=True, ascending=False)
print(Result_tabnet)

     max_epochs  batch_size  virtual_batch_size  patience  Recall
0            20        1024                 128        20     1.0
356          21        1024                 129        26     1.0
1            20        1024                 128        21     1.0
358          21        1024                 130        21     1.0
359          21        1024                 130        22     1.0
..          ...         ...                 ...       ...     ...
199          20        1028                 128        23     0.0
200          20        1028                 128        24     0.0
201          20        1028                 128        25     0.0
202          20        1028                 128        26     0.0
198          20        1028                 128        22     0.0

[452 rows x 5 columns]


In [52]:
Result_tabnet[Result_tabnet['Recall']<0.95]

,max_epochs,batch_size,virtual_batch_size,patience,Recall
448,21,1026,129,20,0.86631
449,21,1026,129,21,0.86631
450,21,1026,129,22,0.86631
451,21,1026,129,23,0.86631
111,20,1026,129,26,0.86631
...,...,...,...,...,...
199,20,1028,128,23,0.00000
200,20,1028,128,24,0.00000
201,20,1028,128,25,0.00000
202,20,1028,128,26,0.00000


In [25]:
Result_tabnet[Result_tabnet['Recall']<0.95]

NameError: name 'Result_tabnet' is not defined

# Modelo luego del primer año

In [47]:
clf_1= TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=1,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-2),
                       scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.95},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
                      )

clf_1.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=1000 , patience=22,
    batch_size=1028, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
) 

Device used : cpu
epoch 0  | loss: 0.71768 | train_auc: 0.495   | valid_auc: 0.50389 |  0:00:03s
epoch 1  | loss: 0.69986 | train_auc: 0.49994 | valid_auc: 0.49983 |  0:00:07s
epoch 2  | loss: 0.6956  | train_auc: 0.47835 | valid_auc: 0.46691 |  0:00:11s
epoch 3  | loss: 0.69407 | train_auc: 0.52276 | valid_auc: 0.53543 |  0:00:15s
epoch 4  | loss: 0.69372 | train_auc: 0.52555 | valid_auc: 0.52692 |  0:00:19s
epoch 5  | loss: 0.69428 | train_auc: 0.52732 | valid_auc: 0.52678 |  0:00:22s
epoch 6  | loss: 0.69285 | train_auc: 0.52383 | valid_auc: 0.52945 |  0:00:26s
epoch 7  | loss: 0.69411 | train_auc: 0.52232 | valid_auc: 0.53164 |  0:00:30s
epoch 8  | loss: 0.694   | train_auc: 0.52209 | valid_auc: 0.5253  |  0:00:34s
epoch 9  | loss: 0.69333 | train_auc: 0.51538 | valid_auc: 0.51081 |  0:00:37s
epoch 10 | loss: 0.69484 | train_auc: 0.47343 | valid_auc: 0.46199 |  0:00:41s
epoch 11 | loss: 0.6934  | train_auc: 0.4734  | valid_auc: 0.46964 |  0:00:45s
epoch 12 | loss: 0.69325 | train_a

In [48]:
Report = classification_report(y_test,clf_1.predict(X_test),labels=[0,1])
print('Classification report : \n',Report)

Classification report : 
               precision    recall  f1-score   support

           0       0.44      0.75      0.55       778
           1       0.61      0.29      0.39      1055

    accuracy                           0.48      1833
   macro avg       0.52      0.52      0.47      1833
weighted avg       0.54      0.48      0.46      1833



In [49]:
dataset_name = 'data_pre'
preds = clf_1.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)
print(f"AUC : {test_auc}")

AUC : 0.5252927058078193


In [50]:
pd.crosstab(clf_1.predict(X_test), y_test, 
            rownames=['pred'], 
            colnames=['Actual'], margins=False, margins_name="Total")

Actual,0,1
pred,,
0,584,753
1,194,302


In [51]:
classification_report(y_test,clf_1.predict(X_test),labels=[0,1], output_dict=True)['0']['recall']

0.7506426735218509

In [52]:
classification_report(y_test,clf_1.predict(X_test),labels=[0,1], output_dict=True)['0']['f1-score']

0.5522458628841608

In [53]:
preds = clf_1.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)
print(f"AUC : {test_auc}")

AUC : 0.5252927058078193


In [56]:
194/(194+584+753+302)

0.10583742498636116